<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Premortem Decision Analysis  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (Pre-Mortem)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem a planning exercise where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential problems and fix them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the project stronger.  

### Example Premortem Scenario  

Imagine you're about to buy a car. In a pre-mortem, you pretend the purchase turned out to be a disaster. Now, ask yourself: What went wrong?

Here are some potential reasons:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

By imagining these issues in advance, you could decide to thoroughly check maintenance records, test drive more carefully, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" outcomes and also brainstorm potential reasons why that certain outcome occured. 